# Manual adjustment

With this notebook, you can manually do CIGALE works.  There's a lot of code cell but the interesting part is the interactive plot at the end.  This plot allows you to select one of the KINGFISH galaxy of the fitting execise.  You will then see on the plot:

- the spectrum create by CIGALE with the parameters to the left
- the fluxes of this CIGALE SED computed in the bands of the KINGFISH catalogue
- the fluxes of the selected galaxy taken from the catalogue

Compared to the CIGALE introduction notebook, there is one more parameter that is the **adjustment factor** (well, in fact it's log10).  By default, CIGALE computes the spectrum for one solar mass produced during the Star Formation History.  Then, the spectrum must be adjusted for the actual mass of the analysed galaxy.

Just do `Kernel → Restart kernel and run all cells` and go to the end of the page.

In [ ]:
%matplotlib ipympl
import matplotlib
matplotlib.rcParams['figure.figsize'] = [9.,6.]

In [ ]:
import numpy as np

from IPython.display import display
from ipywidgets import widgets, AppLayout
from matplotlib import pyplot as plt

from astropy.table import Table

from pcigale import sed
from pcigale import sed_modules as modules
from pcigale.warehouse import SedWarehouse
from pcigale.data import SimpleDatabase

In [ ]:
KINGFISH = Table.read("../exercise/Sings_KINGFISH_sample.txt", format='ascii')[:-1]  # remove dummy_ga
KINGFISH.add_index('id')

In [ ]:
BANDS = [
    col for col in KINGFISH.colnames if not col.endswith("_err") 
    and not col.startswith("line") 
    and not col in ["id", "redshift", "distance"]
]

d = SimpleDatabase("filters")
BANDS_WAVE = np.array([d.get(name=name).pivot for name in BANDS])

In [ ]:
# Initial values of the SED parameters
SED_PARAMETERS = {
    'sfhdelayed': {
        'tau_main': 1000.,
        'age_main': 8000.,
        'tau_burst': 2000.,
        'age_burst': 50.,
        'f_burst': 0.05,
        },
    'bc03': {
        'imf': 0,
        'metallicity': 0.02,
    },
    'nebular': {
        'logU': -2.0,
        'f_esc': 0.0,
        'f_dust': 0.0,
        'emission': True,
    },
    'dustatt_modified_starburst': {
        'E_BV_lines': 0.08,
        'E_BV_factor': 1,
        'uv_bump_wavelength': 217.5,
        'uv_bump_width': 35.0,
        'uv_bump_amplitude': 0.0,
        'powerlaw_slope': 0.0,
        'Ext_law_emission_lines': 1,
        'Rv': 3.1,
    },
    'dl2014': {
        'qpah': 2.50,
        'umin': 1.5,
        'alpha': 2.0,
        'gamma': 0.02,
    },
    'redshifting': {
        'redshift': 2,
    }
}

In [ ]:
WAREHOUSE = SedWarehouse()

def plot_sed(name, log_adjust_factor, tau_main, age_main, tau_burst, age_burst, f_burst, metallicity, logU,
             f_esc, f_dust, E_BV_lines, E_BV_factor, uv_bump_width, uv_bump_amplitude, powerlaw_slope, Rv, qpah, 
             umin, alpha, gamma, redshift, l_range):
    plt.figure("Interactive_SED", figsize=(9, 7))
    sed = WAREHOUSE.get_sed(
        module_list = ['sfhdelayed', 'bc03', 'nebular', 'dustatt_modified_starburst', 'dl2014', 'redshifting'],
        parameter_list= [
            {
                'tau_main': tau_main,
                'age_main': age_main,
                'tau_burst': tau_burst,
                'age_burst': age_burst,
                'f_burst': f_burst,
            },{
                'imf': 0,
                'metallicity': metallicity,
            },{
                'logU': logU,
                'f_esc': f_esc,
                'f_dust': f_dust,
                'emission': True,
            },{
                'E_BV_lines': E_BV_lines,
                'E_BV_factor': E_BV_factor,
                'uv_bump_wavelength': 217.5,
                'uv_bump_width': uv_bump_width,
                'uv_bump_amplitude': uv_bump_amplitude,
                'powerlaw_slope': powerlaw_slope,
                'Ext_law_emission_lines': 1,
                'Rv': Rv,
            },{
                'qpah': qpah,
                'umin': umin,
                'alpha': alpha,
                'gamma': gamma,
            }, {
                'redshift': redshift,
            }
        ]
    )
    adjust_factor = 10**log_adjust_factor
    band_sed_fluxes = np.array([adjust_factor * sed.compute_fnu(band) for band in BANDS])
    band_gal_fluxes = np.array([KINGFISH.loc[name][band] for band in BANDS])
    band_gal_err = [KINGFISH.loc[name][f"{band}_err"] for band in BANDS]
    x_lims = (10**l_range[0], 10**l_range[1])
    plt.clf()
    plt.grid()
    plt.loglog(sed.wavelength_grid, adjust_factor * sed.fnu, label="SED")
    plt.scatter(BANDS_WAVE, band_sed_fluxes, label="SED band fluxes")
    plt.errorbar(BANDS_WAVE, band_gal_fluxes, yerr=band_gal_err, fmt='.', label=f"{name} fluxes")
    plt.xlim(x_lims)
    # Recompute the y limits
    mask_sed = (sed.wavelength_grid >= x_lims[0]) & (sed.wavelength_grid <= x_lims[1])
    mask_bands = (BANDS_WAVE >= x_lims[0]) & (BANDS_WAVE <= x_lims[1])
    y_min = 0.9 * np.min([np.min(adjust_factor * sed.fnu[mask_sed]), np.min(band_sed_fluxes[mask_bands]), np.min(band_gal_fluxes[mask_bands])])
    if y_min <= 0:
        y_min = 1e-40  # log axis
    y_max = 1.1 * np.max([np.max(adjust_factor * sed.fnu[mask_sed]), np.max(band_sed_fluxes[mask_bands]), np.max(band_gal_fluxes[mask_bands])])
    plt.ylim((y_min, y_max))
    plt.legend(loc=0)
    plt.ylabel ("Flux [Jy]")
    plt.xlabel("Wavelength [nm]")
    #plt.tight_layout()
    _ = plt.show()

In [ ]:
plt.close("Interactive_SED")  # needed to rerun the cell

name = widgets.Dropdown(value=KINGFISH['id'][0], 
                        options=list(KINGFISH['id']))

log_adjust_factor = widgets.FloatSlider(11, min=0, max=16, step=0.1)

tau_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_main'], 
                               min=500, max=8000, step=100, 
                               description="tau_main", continuous_update=False)
age_main = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_main'], 
                               min=2000, max=10000, step=1000, 
                               description="age_main", continuous_update=False)
tau_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['tau_burst'], 
                                min=500, max=40000, step=100, 
                                description="tau_burst", continuous_update=False)
age_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['age_burst'],
                                min=50, max=500, step=100, 
                                description="age_burst", continuous_update=False)
f_burst = widgets.FloatSlider(value=SED_PARAMETERS['sfhdelayed']['f_burst'], 
                              min=0, max=.9, step=.1, 
                              description="f_burst", continuous_update=False)

metallicity = widgets.Dropdown(value=SED_PARAMETERS['bc03']['metallicity'], 
                               options=[0.0001, 0.0004, 0.004, 0.008, 0.02, 0.05], 
                               description="metallicity")

logU = widgets.Dropdown(value=SED_PARAMETERS['nebular']['logU'],
                        options=[-4.0, -3.9, -3.8, -3.7, -3.6, -3.5, -3.4, -3.3, 
                                 -3.2, -3.1, -3.0, -2.9, -2.8, -2.7, -2.6, -2.5, 
                                 -2.4, -2.3, -2.2, -2.1, -2.0, -1.9, -1.8, -1.7,
                                 -1.6, -1.5, -1.4, -1.3, -1.2, -1.1, -1.0],
                        description="logU")
f_esc = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_esc'], 
                            min=0, max=1, step=.1, 
                            description="f_esc", continuous_update=False)
f_dust = widgets.FloatSlider(value=SED_PARAMETERS['nebular']['f_dust'], 
                             min=0, max=1, step=.1, 
                             description="f_dust", continuous_update=False)
# f_esc + f_dust is at most 1 (the remaining part is emitted as lines)
def update_f_esc(*args):
    if f_esc.value + f_dust.value > 1:
        f_esc.value = 1 - f_dust.value
f_dust.observe(update_f_esc, 'value')
def update_f_dust(*args):
    if f_esc.value + f_dust.value > 1:
        f_dust.value = 1 - f_esc.value
f_esc.observe(update_f_dust, 'value')

E_BV_lines = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_lines'],
                                 min=0, max=1, step=.05,
                                 description='E_BV_lines', continuous_update=False)
E_BV_factor = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['E_BV_factor'],
                                 min=0, max=1, step=.1,
                                 description='E_BV_factor', continuous_update=False)
uv_bump_amplitude = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_amplitude'], 
                                        min=0, max=5, step=0.1, 
                                        description="bump_ampl.", continuous_update=False)
uv_bump_width = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['uv_bump_width'],
                                    min=100, max=500, step=100, 
                                    description="bump_width", continuous_update=False)

powerlaw_slope  = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['powerlaw_slope'],
                                      min=-1, max=1, step=.1, 
                                      description="slope", continuous_update=False)
Rv = widgets.FloatSlider(value=SED_PARAMETERS['dustatt_modified_starburst']['Rv'],
                         min=0, max=5, step=.1, 
                         description="Rv", continuous_update=False)

qpah = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['qpah'],
                        options=[0.47, 1.12, 1.77, 2.50, 3.19, 3.90, 4.58, 5.26, 5.95, 
                                 6.63, 7.32],
                        description="qpah")
umin = widgets.Dropdown(value=SED_PARAMETERS['dl2014']['umin'],
                        options=[0.100, 0.120, 0.150, 0.170, 0.200, 0.250, 0.300, 0.350, 
                                 0.400, 0.500, 0.600, 0.700, 0.800, 1.000, 1.200, 1.500, 
                                 1.700, 2.000, 2.500, 3.000, 3.500, 4.000, 5.000, 6.000, 
                                 7.000, 8.000, 10.00, 12.00, 15.00, 17.00, 20.00, 25.00, 
                                 30.00, 35.00, 40.00, 50.00],
                        description="umin")
alpha = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['alpha'],
                            min=1., max=3., step=.5, 
                            description="alpha", continuous_update=False)
gamma = widgets.FloatSlider(value=SED_PARAMETERS['dl2014']['gamma'],
                            min=0., max=20., step=1, 
                            description="gamma", continuous_update=False)

redshift = widgets.FloatSlider(value=SED_PARAMETERS['redshifting']['redshift'],
                               min=0, max=10, step=.1, 
                               description="redshift", continuous_update=False)

l_range = widgets.FloatRangeSlider(value=(1.5, 6), min=1, max=8, step=.1, continuous_update=False)

sliders = widgets.VBox([
    widgets.Text("Lambda range (log)"),
    l_range,
    widgets.Text("Kingfish galaxy name"),
    name,
    widgets.Text("Log of adjust factor"),
    log_adjust_factor,
    widgets.Text("Star Formation History"),
    tau_main, age_main, tau_burst, age_burst, f_burst,
    widgets.Text("BC03 SSP"),
    metallicity, 
    widgets.Text("Nebular"),
    logU, f_esc, f_dust, 
    widgets.Text("Dust Attenuation"),
    E_BV_lines, E_BV_factor, uv_bump_amplitude, uv_bump_width, powerlaw_slope, Rv, 
    widgets.Text("IR emission"),
    qpah, umin, alpha, gamma, 
    widgets.Text("Redshift"),
    redshift,
])

figure = widgets.interactive_output(
    plot_sed,
    {
        'name': name,
        'log_adjust_factor': log_adjust_factor,
        'tau_main': tau_main,
        'age_main': age_main,
        'tau_burst': tau_burst,
        'age_burst': age_burst,
        'f_burst': f_burst,
        'metallicity': metallicity,
        'logU': logU,
        'f_esc': f_esc,
        'f_dust': f_dust,
        'E_BV_lines': E_BV_lines,
        'E_BV_factor': E_BV_factor,
        'uv_bump_amplitude': uv_bump_amplitude,
        'uv_bump_width': uv_bump_width,
        'powerlaw_slope': powerlaw_slope,
        'Rv': Rv,
        'qpah': qpah,
        'umin': umin,
        'alpha': alpha,
        'gamma': gamma,
        'redshift': redshift,
        'l_range': l_range
    }
)

In [ ]:
AppLayout(header=None,
          left_sidebar=None,
          center=figure,
          right_sidebar=sliders,
          footer=None)